In [ ]:
# load libraries and scripts
include("../scripts/tools.jl")
include("../scripts/plotting.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")
include("../scripts/disturbances.jl");
include("../scripts/ps_analysis.jl");

In [ ]:
#load borders
border, scale_factor = import_border("../data/borders/border.json")

dx = 0.005
Nx, Ny, coord, isinside, isborder, isgrid, n = get_grid(border, dx);

In [ ]:
# compute or load grid parameters
minv, gamma, p, xi, bx, by, m, d = get_params(isinside, isgrid, n, Ny, Nx, Float64(dx), coord,
    scale_factor, "../data/disc/pantagruel.h5",
    "../numerics/grid_params_" * string(dx) * ".h5",
    Niter=10, dmax = 2*dx, patch=4000.0, bmin=0.0, tau=0.00001);

In [ ]:
th0 = compute_stable_sol(isinside, n, bx, by, p, interval = 10000, Niter = 60000, tol = 1e-7);

In [ ]:
gen, dem, bline, idb, idgen, coord_disc, mg, dg, dl, th_ref = load_discrete_model("../data/disc/pantagruel.h5", scale_factor)

In [ ]:
# add a disturbance
#dp = local_disturbance(coord[isgrid,:], [-0.2; -0.5], -10.0, 0.1, dx)
add_local_disturbance!(contmod, [-0.35; 0.24], -10.0, 0.02, dx)

In [ ]:
#temp = zeros(size(isgrid))
#temp[isgrid] = dp
#heatmap(reshape(temp,Ny,Nx))

In [ ]:
ts, ~, omegas = perform_dyn_sim(isgrid, xi, p+dp, minv, gamma, th0, interval=10,
    Ndt=5000, dt=0.01, method="crank-nicolson");

In [ ]:
c = [500 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000; -1000 -400] / scale_factor
time_plot(ts, omegas, coord[isgrid,:], c, borders = [border], tstart = 0.0, tend = 50.0)

In [ ]:
data = h5read("../numerics/disc_results.h5","/");
o = data["omegas"];
t = data["ts"];
println(size(t))
println(size(o))
reshape(t,prod(size(o)),1)
t[1:size(o,2)]

In [ ]:
#c = [500 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000; -1000 -400] / scale_factor
c = [-.30 -.25; 0.0 0.0; -.5 -.30; .25 .15]

cmp_plot( coord[isgrid,:], omegas, ts, coord_disc[idgen[gen .> 0.0],:], o[:,1:end-1], t[1:size(o,2)-1],
    c, tstart=0.0, tend=20.0)
current()